# Workflow Examples

This chapter demonstrates three common workflow patterns using the nodes from the previous chapter. Import the pre-built workflows, run them, and observe how the nodes work together.

**Workflow files:** The JSON files are in `courses/n8n_no_code/01_intro_patterns/workflows/`

**Note on credentials:** These workflows require API credentials for an AI provider (OpenRouter, OpenAI, or Google). If you do not have credentials set up, you can still explore the workflow structure and understand what each node would do.

---

## How to Import and Run a Workflow

**Import:**
1. In n8n, click **Workflows** in the left sidebar
2. Click **Add Workflow**
3. Click the **three-dot menu (⋮)** in the top-right
4. Select **Import from File**
5. Choose the `.json` file
6. Click **Save**

**Run:**
1. Open the workflow in the editor
2. Make sure credentials are set up (Settings → Credentials)
3. Click **Execute Workflow** in the top toolbar
4. Click any node to see its output in the right panel

---

## Pattern 1: Prompt Chaining

**File:** `01_prompt_chaining.json`

### What This Pattern Does

Breaks a complex task into sequential LLM calls, where each call builds on the previous result.

**Structure:**
```
Trigger → Set (input) → LLM 1 → Set (save) → LLM 2 → Set (save) → LLM 3
```

### Nodes in This Workflow

| Node | What it does |
|------|-------------|
| Manual Trigger | Starts the workflow |
| Input (Set) | Defines topic, audience, constraints |
| Chat Model | Provides the AI model |
| Step 1: Create Outline | LLM generates an outline |
| Store Outline (Set) | Saves the outline with a clear name |
| Step 2: Improve Outline | LLM refines the outline |
| Store Improved (Set) | Saves the improved version |
| Step 3: Draft Document | LLM writes the final document |

### What to Observe

- **Input node:** See the starting data (topic, audience, constraints)
- **Each LLM node:** Click to see the prompt sent and response received
- **Each Set node:** See how intermediate results are saved with clear field names
- **Final output:** The completed document in the `text` field

### Why Use Checkpoints (Set Nodes)?

The Set nodes between LLM calls serve as "checkpoints":
- Save each result with a descriptive name (`outline`, `improved_outline`)
- Make debugging easier — you can see exactly what each step produced
- Allow pinning at specific steps to avoid re-running expensive LLM calls

### Try This

Change the `topic` field in the first Set node to a different subject. Run the workflow again and observe how the entire chain produces different output.

---

## Pattern 2: Routing

**File:** `02_routing.json`

### What This Pattern Does

Uses an LLM to classify input, then routes data to different branches based on the classification.

**Structure:**
```
Trigger → Set (input) → LLM (classify) → Set (route) → Switch → [Branch A]
                                                              → [Branch B]
                                                              → [Fallback]
```

### Nodes in This Workflow

| Node | What it does |
|------|-------------|
| Manual Trigger | Starts the workflow |
| Input (Set) | Defines the input to classify (e.g., support ticket) |
| Chat Model | Provides the AI model |
| Classifier (LLM) | Outputs a classification label |
| Store Route (Set) | Saves the label in a `route` field |
| Switch | Routes data based on `{{ $json.route }}` |
| Branch nodes | Different processing for each route |

### How the Classifier Works

The classifier LLM prompt explicitly lists allowed labels:

```
Classify this ticket into one of these categories:
- refund
- support
- sales

Output only the category name, nothing else.
```

The LLM outputs a single word (e.g., `refund`), which the Switch node uses to route.

### What to Observe

- **Classifier LLM:** See the prompt and the classification result
- **Set node after classifier:** See the `route` field being saved
- **Switch node:** Click to see which output branch received data (only one will have data)
- **Branch nodes:** Only one branch will have executed

### Try This

Change the input data to trigger a different route. For example, change a refund request to a technical question. Run again and observe which branch activates.

---

## Pattern 3: Parallelization

**File:** `03_parallelization.json`

### What This Pattern Does

Runs multiple LLM calls in parallel, then combines the results.

**Structure:**
```
Trigger → Set (input) → [LLM A: analyze] → Set (save) ─┐
                      → [LLM B: extract] → Set (save) ─┼→ Merge → LLM (final)
                      → [LLM C: draft]   → Set (save) ─┘
```

### Nodes in This Workflow

| Node | What it does |
|------|-------------|
| Manual Trigger | Starts the workflow |
| Input (Set) | Defines the input data |
| Chat Model | Provides the AI model |
| Branch A: LLM | Performs analysis A (e.g., sentiment) |
| Branch A: Set | Saves result as `sentiment` |
| Branch B: LLM | Performs analysis B (e.g., extract facts) |
| Branch B: Set | Saves result as `facts` |
| Branch C: LLM | Performs analysis C (e.g., draft reply) |
| Branch C: Set | Saves result as `draft` |
| Merge | Combines all results into one item |
| Final LLM | Uses combined data to produce final output |

### Why Use Parallel Branches?

- **Speed:** Parallel branches run at the same time, not sequentially
- **Independence:** Each analysis can focus on one thing
- **Modularity:** Easy to add or remove branches

### How the Merge Node Works

With "Combine by Position" mode:
- Each branch outputs one item
- Merge combines all fields into one item:

```json
{
  "sentiment": "positive",
  "facts": ["fact1", "fact2"],
  "draft": "Here is a draft reply..."
}
```

### What to Observe

- **Parallel branches:** Multiple LLM nodes that run at the same time
- **Set nodes in each branch:** Each saves its result with a unique field name
- **Merge node:** Click to see all fields combined (switch to JSON view)
- **Final LLM:** Uses data from all branches in its prompt

### Try This

Click the Merge node and switch to JSON view. Verify that all fields from all branches are present. Then click the final LLM and check that its prompt references data from each branch.

---

## AI Agent Node (Minimal Intro)

The **AI Agent** node is a workflow step that can **decide what to do next** instead of following a single fixed prompt.

- **Tools:** optional actions the agent can call (e.g., calculator, HTTP request, search). Think: "abilities".
- **Memory:** optional context the agent can carry across steps so it stays consistent.
- **Loop (plan → act → check):** the agent can iterate: decide a plan, use a tool (act), check results, and stop when done.

**Tip:** Start simple—use **one tool** (or none) and keep runs in **manual test mode** to avoid accidental costs.

See the **AI Agent Examples** chapter for a hands-on walkthrough.

---

## Pattern Summary

| Pattern | When to use | Key nodes |
|---------|-------------|------------|
| **Prompt Chaining** | Complex tasks that benefit from step-by-step refinement | Multiple LLM Chains in sequence |
| **Routing** | Different handling based on input type | LLM (classifier) + Switch |
| **Parallelization** | Independent analyses that can run simultaneously | Multiple branches + Merge |

---

## Tips for Building Your Own Workflows

1. **Start simple.** Build one node at a time and test frequently.

2. **Use descriptive node names.** "Step 1: Create Outline" is better than "LLM Chain."

3. **Pin data often.** After any successful LLM call, pin the result before working on the next node.

4. **Check the Output panel.** Switch between Table and JSON views to understand your data.

5. **Use Set nodes as checkpoints.** Save intermediate results with clear field names.

6. **Test routing with different inputs.** Make sure all branches work, not just the happy path.

---

## Glossary

| Term | Definition |
|------|------------|
| **Workflow** | A sequence of connected nodes that automate a task |
| **Trigger** | The first node; defines what starts the workflow |
| **Node** | A single step in a workflow |
| **Connection** | Solid = data flow; Dotted = capability |
| **Execution** | One complete run of a workflow |
| **Item** | One unit of data with `.json` property |
| **Expression** | Dynamic value using `{{ $json.field }}` |
| **Pinned Data** | Saved output to reuse without re-running |
| **Credential** | Securely stored API key |

---

## Official Documentation Links

| Topic | Link |
|-------|------|
| Workflows | [docs.n8n.io/workflows](https://docs.n8n.io/workflows/) |
| Executions | [docs.n8n.io/workflows/executions](https://docs.n8n.io/workflows/executions/) |
| Data Pinning | [docs.n8n.io/data/data-pinning](https://docs.n8n.io/data/data-pinning/) |
| Expressions | [docs.n8n.io/code/expressions](https://docs.n8n.io/code/expressions/) |
| Manual Trigger | [docs.n8n.io/.../manualtrigger](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.manualtrigger/) |
| Edit Fields (Set) | [docs.n8n.io/.../set](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.set/) |
| AI Nodes | [docs.n8n.io/.../cluster-nodes](https://docs.n8n.io/integrations/builtin/cluster-nodes/) |
| Basic LLM Chain | [docs.n8n.io/.../chainllm](https://docs.n8n.io/integrations/builtin/cluster-nodes/root-nodes/n8n-nodes-langchain.chainllm/) |
| Switch | [docs.n8n.io/.../switch](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.switch/) |
| Merge | [docs.n8n.io/.../merge](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.merge/) |